In [ ]:
import findspark
findspark.init('/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max, desc
from pyspark.sql.functions import grouping, explode, array_contains
from pyspark.sql.functions import mean, stddev, skewness, kurtosis
import matplotlib.pyplot as plt
import pandas as pd
import random


spark = SparkSession.builder.appName("analytics").getOrCreate()

# Análisis de Bitácoras

## Generar las bitácoras

Creamos un archivo de bitácora para un sitio web de ecommerce con visitas a 10 páginas de productos en formato Apache httpd. (Apache httpd logs)[https://httpd.apache.org/docs/2.4/logs.html]

In [ ]:
from datetime import datetime, timedelta

# Generar el archivo de bitácora
num_entries = 100
product_pages = [f"/product/{i}" for i in range(1, 11)]
start_time = datetime(2024, 7, 18, 10, 0, 0)

with open('ecommerce_log.txt', 'w') as log_file:
    for _ in range(num_entries):
        time_offset = timedelta(seconds=random.randint(0, 3600))
        log_time = start_time + time_offset
        log_time_str = log_time.strftime('%d/%b/%Y:%H:%M:%S +0000')
        product_page = random.choice(product_pages)
        response_time = random.randint(100, 5000)  
        log_entry = f'127.0.0.1 - - [{log_time_str}] "GET {product_page} HTTP/1.1" 200 2326 {response_time}\n'
        log_file.write(log_entry)


## Procesamiento de la bitácora

Ahora procesamos la bitácora utilizando Spark

In [ ]:
# Cargar el archivo de bitácora que creamos anteriormente
log_file = "ecommerce_log.txt"
logs = spark.read.text(log_file)

logs.show(5)

In [ ]:
# Definir patrones para extraer los campos de interés, incluyendo el tiempo de respuesta
log_pattern = r'(\d+\.\d+\.\d+\.\d+) - - \[(.+)\] "GET (.+) HTTP/1\.1" (\d+) (\d+) (\d+)'

# Extraer los campos usando expresiones regulares
logs_df = logs.select(
    regexp_extract('value', log_pattern, 1).alias('ip'),
    regexp_extract('value', log_pattern, 2).alias('timestamp'),
    regexp_extract('value', log_pattern, 3).alias('request'),
    regexp_extract('value', log_pattern, 4).alias('status'),
    regexp_extract('value', log_pattern, 5).alias('bytes'),
    regexp_extract('value', log_pattern, 6).alias('response_time').cast('int')
)

# Filtrar solo las solicitudes de productos
product_logs_df = logs_df.filter(col('request').startswith('/product'))

product_logs_df.show(5)

In [ ]:
# Contar las visitas por producto y calcular el tiempo de respuesta promedio
product_counts = product_logs_df.groupBy('request').agg(
    {'*': 'count', 'response_time': 'avg'}
).withColumnRenamed('count(1)', 'visit_count').withColumnRenamed('avg(response_time)', 'avg_response_time')

# Mostrar el resultado
product_counts.show()

## Visualización de bitácoras

In [ ]:
import matplotlib.pyplot as plt

# Convertir los datos a Pandas para su visualización
product_counts_pd = product_counts.toPandas()

# Crear la gráfica de visitas por producto
plt.figure(figsize=(12, 6))

# Gráfico de barras para el número de visitas
plt.subplot(1, 2, 1)
plt.bar(product_counts_pd['request'], product_counts_pd['visit_count'], color='blue')
plt.xlabel('Producto')
plt.ylabel('Número de Visitas')
plt.title('Visitas por Producto en el Sitio de Ecommerce')
plt.xticks(rotation=45)



In [ ]:
# Gráfico de barras para el tiempo de respuesta promedio
plt.subplot(1, 2, 2)
plt.bar(product_counts_pd['request'], product_counts_pd['avg_response_time'], color='green')
plt.xlabel('Producto')
plt.ylabel('Tiempo de Respuesta Promedio (ms)')
plt.title('Tiempo de Respuesta Promedio por Producto')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Convertir los datos a Pandas para su visualización
logs_pd = product_logs_df.toPandas()

# Convertir la columna de timestamp a formato datetime
logs_pd['timestamp'] = pd.to_datetime(logs_pd['timestamp'], format='%d/%b/%Y:%H:%M:%S +0000')

# Ordenar los datos por timestamp
logs_pd = logs_pd.sort_values('timestamp')

# Crear la gráfica de tiempo de respuesta en el tiempo
plt.figure(figsize=(12, 6))
plt.plot(logs_pd['timestamp'], logs_pd['response_time'], marker='o', linestyle='-', color='blue')
plt.xlabel('Tiempo')
plt.ylabel('Tiempo de Respuesta (ms)')
plt.title('Tiempo de Respuesta del Servidor en el Tiempo')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()